# まとめ
- sigmoid関数とtanh関数の逆伝搬の式
    - 微分を求めるには，計算グラフによる方法と解析的な方法がある
- WordNetを動かす
    - PythonからWordNetを動かすには，nltk(Natural Language Toolkit)パッケージをインストールして使う
    - 単語の意味ネットワークが構築されており，様々な単語間類似度の手法が提供されている
- GRU(Gated Recurrent Unit)
    - LSTMよりもパラメータ数が少なく，高速なRNN
    - LSTMとの精度差は，タスクやハイパーパラメータの調整によって様々．どちらが良いとは言い切れない．

# 付録A sigmoid関数とtanh関数の微分

## sigmoid関数
sigmoid関数は次の式で表される
$$ y = \frac{1}{1 + exp(-x)} $$
計算グラフは次のような流れになる．  

<div style="text-align: center">(x, -1) -> [×] ->(-x) -> [exp] -> (exp(-x), 1) -> [+] -> 1+exp(-x) -> [/] -> 1/(1+exp(-x)) = y -></div>
[×]と[+]はすでに扱ったが，[exp]と[/]の微分はまだ説明されていない．  

### /ノードの逆伝搬
逆数を取るノード
$$ y = \frac{1}{x} $$
$x$で偏微分すると，商の微分法から次のようになる
$$ \frac{\partial y}{\partial x} = \frac{0 - 1}{x^2} = -\Bigl(\frac{1}{x}\Bigr)^2 = -y^2 $$
よって/ノードでは，上流からの勾配$\frac{\partial L}{\partial y}$に対して，出力した$y$をキャッシュしておいて以下のように返せばよい．  
$$ -\frac{\partial L}{\partial y}y^2$$

### expノードの逆伝搬
自然対数の累乗を取るノード
$$ y = e^x $$
微分しても$e^x$である．  
$$ \frac{\partial y}{\partial x} = e^x = y $$
よってexpノードでは，上流からの勾配$\frac{\partial L}{\partial y}$に対して，出力した$y$をキャッシュしておいて以下のように返せばよい．  
$$ \frac{\partial L}{\partial y} y $$

### sigmoid関数全体の逆伝搬
計算グラフをたどり，sigmoid関数は出力$y$をキャッシュしておいて以下のように返せばよい．  
$$ \begin{eqnarray*} \\
    \frac{\partial L}{\partial y} (-y_{div}^2)(y_{exp})(-1) &=& \frac{\partial L}{\partial y} \frac{1}{(1 + \exp(-x))^2} \exp(-x) \\
    &=& \frac{\partial L}{\partial y} \frac{1}{1 + \exp(-x)} \frac{exp(-x)}{1 + \exp(-x)} \\
    &=& \frac{\partial L}{\partial y} \frac{1}{1 + \exp(-x)} \Big(1 - \frac{1}{1 + \exp(-x)} \Big) \\
    &=& \frac{\partial L}{\partial y} y(1-y) \\
\end{eqnarray*} $$

## A2. tanh関数
5章を参照  
$$ y = \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$$
の微分を求める．商の微分法を使って
$$ \begin{eqnarray*} \\
    \frac{\partial \tanh(x)}{\partial x} &=& \frac {({e^x + e^{-x}})({e^x + e^{-x}}) - ({e^x - e^{-x}})({e^x - e^{-x}})}{({e^x + e^{-x}})^2} \\
    &=& 1 - \Bigl\{ \frac {({e^x - e^{-x}})}{({e^x + e^{-x}})} \Bigr\}^2 \\
    &=& 1 - \tanh(x)^2 \\
    &=& 1 - y^2
\end{eqnarray*} $$
よって, 上流からの勾配$\frac{\partial L} {\partial y}$を使って
$$ \frac{\partial L} {\partial x} = \frac{\partial L} {\partial y} \cdot \frac{\partial y} {\partial x} = \frac{\partial L} {\partial y} (1 - y^2) $$
を下流に返せば良い．  
ここでyは順伝播の時に保持しておく．RNNレイヤの実装でいうh_next， 最終的な出力を使うことになる．

## A3 まとめ
NNの微分は計算グラフで解く方法(A1)と解析的な方法で解く方法(A2)がある．  
問題に応じて適宜どちらかを利用する．  
複数の方法で問題を解決できることは，時としてとても重要である．

# 付録B WordNetを動かす
WordNetをPythonから利用するには，**NLTK**(Natural Language Toolkit)というライブラリを使う  
NLTKには，自然言語処理のための便利な機能が多く用意されている．
- 品詞タグ付け
- 構文解析
- 情報抽出
- 意味解析
NLTKをインストールするにはpipを使う  
<div style="text-align: center">$ pip install nltk</div>


In [1]:
import nltk# 重い

nltkを使うには，データのダウンロードを行う必要がある．

In [7]:
# nltk.download('wordnet', download_dir="D:\\Programming\\machinelearning\\nltk") # ダウンロード先を適宜変更すること

[nltk_data] Downloading package wordnet to
[nltk_data]     D:/Programming/machinelearning/nltk...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [3]:
# nltkのデータが入っているパスを追加する必要がある．
nltk.data.path.append('D:\\Programming\\machinelearning\\nltk')

## B2 Wordnetで同義語を得る
carという単語について同義語を取得する  
<br>
まずはcarという単語にどれだけ異なる意味が存在するのかを確認する．  
wordnetでは各単語がsynsetと呼ばれる同義語のグループに分類されている．  
同義語を取得するにあたっては，それら複数の意味の中から，どの意味に該当するものかを指定する必要がある．

In [5]:
from nltk.corpus import wordnet
wordnet.synsets('car')

[Synset('car.n.01'),
 Synset('car.n.02'),
 Synset('car.n.03'),
 Synset('car.n.04'),
 Synset('cable_car.n.01')]

バージョン変わった？

carという単語には5つの意味のグループがあることが分かる．  
それぞれについている見出し語は，たとえば「car.n.01」なら，carの「noun(名詞)」の1番目の意味のグループであることを意味する．  
WordNetのメソッドで引数に単語名を指定する場合，「car」だけでなく「car.n.01」のような見出し語で指定する場合が多い．  
<br>
carという単語の意味を確認してみる．

In [6]:
car = wordnet.synset('car.n.01') # 見出し語を指定するときはsynsetsではなくsynset
car.definition()

'a motor vehicle with four wheels; usually propelled by an internal combustion engine'

このcarグループにはどのような同義語が存在するのか確認してみよう．  

In [7]:
car.lemma_names()

['car', 'auto', 'automobile', 'machine', 'motorcar']

## B3 WordNetと単語ネットワーク
他の単語との意味的な上位，下位の関係性について調べる．

In [8]:
car.hypernym_paths()[0] # hypernym: 上位語という意味，

# [0]のように，リストが返されているのは，carに到着するまでの経路が複数存在するから．

[Synset('entity.n.01'),
 Synset('physical_entity.n.01'),
 Synset('object.n.01'),
 Synset('whole.n.02'),
 Synset('artifact.n.01'),
 Synset('instrumentality.n.03'),
 Synset('container.n.01'),
 Synset('wheeled_vehicle.n.01'),
 Synset('self-propelled_vehicle.n.01'),
 Synset('motor_vehicle.n.01'),
 Synset('car.n.01')]

## B4 WordNetによる意味の類似度
WordNetでは，単語間の意味的なネットワークが構築されており，様々な問題で有用である．  
ここでは，単語間の類似度(意味経路の類似度)を求めてみる

In [9]:
novel = wordnet.synset('novel.n.01')
dog = wordnet.synset('dog.n.01')
motorcycle = wordnet.synset('motorcycle.n.01')

print("car : novel", car.path_similarity(novel))
print("car : dog", car.path_similarity(dog))
print("car : motorcycle", car.path_similarity(motorcycle))

car : novel 0.05555555555555555
car : dog 0.07692307692307693
car : motorcycle 0.3333333333333333


他にも，Leacock-Chodorow類似度やWu-Palmer類似度など，いくつかの類似度計測のための手法が用意されている．

# GRU
LSTMはパラメータが多い．  
LSTMに代わるゲート付きRNNとして，**GRU**(Gated Recurrent Unit)を紹介  
GRUはパラメータがLSTMに比べて少ないため，計算時間が短縮できる．

## C1 GRUのインタフェース
LSTMが隠れ状態$h_{t-1}$と記憶セル$c_{t-1}$の2つを受け取るのに対し，GRUは$h_{t-1}$のみを使用する．  

## C2 GRUの計算グラフ
GRUの内部で行われる計算は以下
$$ \begin{eqnarray*} \\
    z &=& \sigma(x_tW_x^{(z)} + h_{t-1}W_h^{(z)} + b^{(z)}) \\
    r &=& \sigma(x_tW_x^{(r)} + h_{t-1}W_h^{(r)} + b^{(r)}) \\
    \widetilde{h} &=& \tanh(x_tW_x + (r \odot h_{t-1})W_h + b) \\
    h_t &=& (1-z) \odot h_{t-1} + z \odot \widetilde{h} \\
\end{eqnarray*} $$
計算グラフは省略  
$r$はリセットゲート，$z$はupdateゲートと呼ばれる．  
<br>
$r$は過去の隠れ状態をどれだけ無視するのかを決定し，新しい隠れ状態$\widetilde{h}$とする．  
$z$は隠れ状態を更新するゲートで，LSTMのforgetゲートとinputゲートの2つの役割を担う．  
$(1-z) \odot h_{t-1}$の部分がforgetゲートの機能，$z \odot \widetilde{h}$はinputゲートとして機能を担う．  
<br>
以上のように，GRUはLSTMをよりシンプルにしたアーキテクチャになる．  
実装はcommon/time_layers.pyにある．

LSTMとGRUでは，タスクやハイパーパラメータの調整によって勝者は変動する．  
最近ではLSTMやLSTMの亜種の方が多く使われつつあるが，GRUも人気を集めている．  
GRUはパラメータ数や計算量が少ないので，データセットのサイズが小さい場合や，モデル設計で繰り返しの施行が必要な場合に適しているといえる．